# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 10 2023 9:03AM,254633,10,Enova-11179,Emerginnova,Released
1,Jan 10 2023 9:03AM,254633,10,Enova-11180,Emerginnova,Released
2,Jan 10 2023 8:53AM,254630,12,HH-39288,Hush Hush,Released
3,Jan 10 2023 8:53AM,254630,12,HH-39289,Hush Hush,Released
4,Jan 10 2023 8:53AM,254630,12,HH-39290,Hush Hush,Released
5,Jan 10 2023 8:53AM,254630,12,HH-39291,Hush Hush,Released
6,Jan 10 2023 8:53AM,254630,12,HH-39292,Hush Hush,Released
7,Jan 10 2023 8:53AM,254630,12,HH-39293,Hush Hush,Released
8,Jan 10 2023 8:53AM,254630,12,HH-39294,Hush Hush,Released
9,Jan 10 2023 8:49AM,254629,10,0086286548,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,254626,Released,1
28,254628,Released,1
29,254629,Released,9
30,254630,Released,7
31,254633,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254626,NaN,NaN,1.0
254628,NaN,NaN,1.0
254629,NaN,NaN,9.0
254630,NaN,NaN,7.0
254633,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254533,2.0,0.0,1.0
254563,37.0,2.0,1.0
254566,0.0,0.0,36.0
254567,0.0,0.0,10.0
254578,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254533,2,0,1
254563,37,2,1
254566,0,0,36
254567,0,0,10
254578,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254533,2,0,1
1,254563,37,2,1
2,254566,0,0,36
3,254567,0,0,10
4,254578,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254533,2,,1
1,254563,37,2,1
2,254566,,,36
3,254567,,,10
4,254578,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 10 2023 9:03AM,254633,10,Emerginnova
2,Jan 10 2023 8:53AM,254630,12,Hush Hush
9,Jan 10 2023 8:49AM,254629,10,ISDIN Corporation
18,Jan 10 2023 8:43AM,254628,20,"Exact-Rx, Inc."
19,Jan 10 2023 8:36AM,254626,10,Bio-PRF
20,Jan 10 2023 8:32AM,254625,20,"Andelyn Biosciences, Inc."
21,Jan 10 2023 7:46AM,254623,19,"AdvaGen Pharma, Ltd"
63,Jan 10 2023 7:40AM,254622,22,"NBTY Global, Inc."
64,Jan 9 2023 4:55PM,254617,15,"Person & Covey, Inc."
67,Jan 9 2023 4:27PM,254616,15,Lonza


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 10 2023 9:03AM,254633,10,Emerginnova,,,2
1,Jan 10 2023 8:53AM,254630,12,Hush Hush,,,7
2,Jan 10 2023 8:49AM,254629,10,ISDIN Corporation,,,9
3,Jan 10 2023 8:43AM,254628,20,"Exact-Rx, Inc.",,,1
4,Jan 10 2023 8:36AM,254626,10,Bio-PRF,,,1
5,Jan 10 2023 8:32AM,254625,20,"Andelyn Biosciences, Inc.",,,1
6,Jan 10 2023 7:46AM,254623,19,"AdvaGen Pharma, Ltd",,1,41
7,Jan 10 2023 7:40AM,254622,22,"NBTY Global, Inc.",,,1
8,Jan 9 2023 4:55PM,254617,15,"Person & Covey, Inc.",,,3
9,Jan 9 2023 4:27PM,254616,15,Lonza,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 9:03AM,254633,10,Emerginnova,2,,
1,Jan 10 2023 8:53AM,254630,12,Hush Hush,7,,
2,Jan 10 2023 8:49AM,254629,10,ISDIN Corporation,9,,
3,Jan 10 2023 8:43AM,254628,20,"Exact-Rx, Inc.",1,,
4,Jan 10 2023 8:36AM,254626,10,Bio-PRF,1,,
5,Jan 10 2023 8:32AM,254625,20,"Andelyn Biosciences, Inc.",1,,
6,Jan 10 2023 7:46AM,254623,19,"AdvaGen Pharma, Ltd",41,1,
7,Jan 10 2023 7:40AM,254622,22,"NBTY Global, Inc.",1,,
8,Jan 9 2023 4:55PM,254617,15,"Person & Covey, Inc.",3,,
9,Jan 9 2023 4:27PM,254616,15,Lonza,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 9:03AM,254633,10,Emerginnova,2,,
1,Jan 10 2023 8:53AM,254630,12,Hush Hush,7,,
2,Jan 10 2023 8:49AM,254629,10,ISDIN Corporation,9,,
3,Jan 10 2023 8:43AM,254628,20,"Exact-Rx, Inc.",1,,
4,Jan 10 2023 8:36AM,254626,10,Bio-PRF,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 9:03AM,254633,10,Emerginnova,2,NaN,NaN
1,Jan 10 2023 8:53AM,254630,12,Hush Hush,7,NaN,NaN
2,Jan 10 2023 8:49AM,254629,10,ISDIN Corporation,9,NaN,NaN
3,Jan 10 2023 8:43AM,254628,20,"Exact-Rx, Inc.",1,NaN,NaN
4,Jan 10 2023 8:36AM,254626,10,Bio-PRF,1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 9:03AM,254633,10,Emerginnova,2,0.0,0.0
1,Jan 10 2023 8:53AM,254630,12,Hush Hush,7,0.0,0.0
2,Jan 10 2023 8:49AM,254629,10,ISDIN Corporation,9,0.0,0.0
3,Jan 10 2023 8:43AM,254628,20,"Exact-Rx, Inc.",1,0.0,0.0
4,Jan 10 2023 8:36AM,254626,10,Bio-PRF,1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1273001,14,0.0,5.0
12,254630,7,0.0,0.0
15,2036800,81,2.0,37.0
16,254585,1,0.0,0.0
19,1527581,55,1.0,0.0
20,509253,2,0.0,0.0
21,763794,3,0.0,0.0
22,254622,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1273001,14,0.0,5.0
1,12,254630,7,0.0,0.0
2,15,2036800,81,2.0,37.0
3,16,254585,1,0.0,0.0
4,19,1527581,55,1.0,0.0
5,20,509253,2,0.0,0.0
6,21,763794,3,0.0,0.0
7,22,254622,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,14,0.0,5.0
1,12,7,0.0,0.0
2,15,81,2.0,37.0
3,16,1,0.0,0.0
4,19,55,1.0,0.0
5,20,2,0.0,0.0
6,21,3,0.0,0.0
7,22,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,14.0
1,12,Released,7.0
2,15,Released,81.0
3,16,Released,1.0
4,19,Released,55.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,5.0,0.0,37.0,0.0,0.0,0.0,0.0,0.0
Executing,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0
Released,14.0,7.0,81.0,1.0,55.0,2.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,5.0,0.0,37.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0
2,Released,14.0,7.0,81.0,1.0,55.0,2.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,5.0,0.0,37.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0
2,Released,14.0,7.0,81.0,1.0,55.0,2.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()